In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()


In [ ]:
import urllib.request
from azureml.core.model import Model

# Download model
from pathlib import Path

my_file = Path("./model.onnx")
if not my_file.exists():
    urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "./model.onnx")


In [ ]:

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)


In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.get_logs())